In [1]:
import pandas as pd
import numpy as np
import os
import glob

## 📌 ใช้ค่า MCV, MCH, Hb เทรน ML เพื่อแยก thalas โดยยึด Fold ตามแบบเดิม

- 🩸แบบ 2 โรค (HN, NBL)

- 🩸 แบบ 3 โรค (Normal, ตัดม้าม, ไม่ตัดม้าม)

### 📂1. DATA : ค่าจากผล Lab

In [4]:
# Read excel file with sheet name
dict_df = pd.read_excel('/home/kannika/code/Rheology2023/Rheology_Blood/All lab results (Blood viscosity work)3.xlsx', 
                   sheet_name=['Patients_ตัดม้าม','Patients_ไม่ตัดม้าม', 'Normal'], header=2)
dict_df

{'Patients_ตัดม้าม':         No.  Code         HN        AGE          PC  PS activity    Free PS  \
 0         1  HN01  2045512.0  29.000000   76.800000    43.000000  58.600000   
 1         2  HN02  4045280.0  21.000000  100.400000    58.800000  70.600000   
 2         3  HN03  3024257.0  41.000000   75.600000    52.500000  57.900000   
 3         4  HN04  1257101.0  40.000000   31.700000    57.700000  59.300000   
 4         6  HN06  1797136.0  32.000000   43.700000    41.100000  77.300000   
 5         7  HN07  2234214.0  50.000000   66.400000    54.500000  62.100000   
 6        12  HN12  1110168.0  45.000000   87.800000    39.300000  60.000000   
 7        13  HN13  3949905.0  23.000000   50.400000    37.000000  83.600000   
 8        16  HN16  1665191.0  35.000000   79.000000    47.400000  59.100000   
 9        17  HN17  2272494.0  27.000000   47.600000    34.200000  59.600000   
 10       19  HN19  1513782.0  36.000000   55.500000    83.000000  76.600000   
 11       23  HN23  

In [41]:
''''
subclass
Patients_ตัดม้าม: Splenectomy
Patients_ไม่ตัดม้าม: No_Splenectomy
Normal
'''

dict_sheet =  ['Patients_ตัดม้าม','Patients_ไม่ตัดม้าม', 'Normal']
lstdict_code  = []
for sheet in dict_sheet:
    df_sheet = dict_df[sheet]
    df_sheet["sheet"] = sheet
    dict_code = df_sheet[['Code','MCV', 'MCH', 'Hb', 'sheet']]
    #dict_code = dict_code[dict_code['Code'].notna()]
    _dict_code = dict_code[(dict_code['Code'].notna())&(dict_code['MCV'].notna())&(dict_code['MCH'].notna())
                         &dict_code['Hb'].notna()]
    __dict_code = _dict_code[(_dict_code['MCV']!= '-')|(_dict_code['MCH']!= '-')
                         |(_dict_code['Hb']!= '-')]
    lstdict_code.append(__dict_code)
    print(dict_df[sheet].shape)

print(len(lstdict_code))
df_dictCode = pd.concat(lstdict_code, ignore_index=True)
df_dictCode["sheet"] = df_dictCode['sheet'].str.replace('Patients_ตัดม้าม','Splenectomy')
df_dictCode["sheet"] = df_dictCode['sheet'].str.replace('Patients_ไม่ตัดม้าม','No_Splenectomy')
print(df_dictCode.shape)
df_dictCode

(27, 53)
(27, 52)
(27, 36)
3
(64, 5)


,Code,MCV,MCH,Hb,sheet
0,HN01,75.8,23.1,6.0,Splenectomy
1,HN02,73.5,22.4,7.2,Splenectomy
2,HN03,77.3,24.3,9.0,Splenectomy
3,HN04,79.8,25.6,7.6,Splenectomy
4,HN06,78.5,25.9,8.2,Splenectomy
...,...,...,...,...,...
59,NBL19,89.5,27.8,14.3,Normal
60,NBL20,95.3,31.7,14.7,Normal
61,NBL21,74.9,23.9,14.5,Normal
62,NBL22,75.1,24.7,15.2,Normal


In [43]:
print(len(list(set(df_dictCode["Code"]))))
print(set(df_dictCode["sheet"]))

64
{'Splenectomy', 'Normal', 'No_Splenectomy'}


In [44]:
df_dictCode.isnull().values.any()

False

In [45]:
df_dictCode.isnull().sum().sum()

0

In [46]:
df_dictCode.to_csv("/home/kannika/code/Rheology2023/Rheology_Blood/All_lab_MCV-MCH-Hb.csv")

## Map Patient Code With Feature(MCV, MCH, Hb)

In [48]:
### ✅ 2 Class => (NBL, HN)
DF_A = pd.read_csv('/home/kannika/code/Rheology2023/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-2dFFTdataset-3channels-6Fold-splitclass.csv')
print(DF_A.shape)
DF_A.head()

(288, 15)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary
0,0,0,0,237,237,HN35_D0_30HZ_20XINF_UWELL_25660131_162433_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162433,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1
1,1,1,1,234,234,HN35_D0_30HZ_20XINF_UWELL_25660131_161754_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_161754,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1
2,2,2,2,324,324,HN43_D0_30HZ_20XINF_UWELL_25660330_153259_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN43_D0_30HZ_20XINF_UWELL_25660330_153259,SSD_Backup,HN,HN43,No_Splenectomy,1,"[2, 5, 7]",1
3,3,3,3,268,268,HN38_D0_30HZ_20XINF_UWELL_25660223_145146_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN38_D0_30HZ_20XINF_UWELL_25660223_145146,SSD_Backup,HN,HN38,No_Splenectomy,1,"[2, 5, 7]",1
4,4,4,4,239,239,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1


In [49]:
Code_A = list(set(DF_A["Code"]))
print(len(Code_A))
Code_A

53


['HN21',
 'HN18',
 'HN25',
 'NBL11',
 'NBL17',
 'HN19',
 'HN41',
 'HN15',
 'HN39',
 'HN27',
 'NBL15',
 'NBL00',
 'HN17',
 'HN30',
 'HN40',
 'HN16',
 'NBL21',
 'HN14',
 'HN02',
 'HN42',
 'NBL13',
 'HN43',
 'HN33',
 'NBL14',
 'HN23',
 'HN32',
 'HN28',
 'HN20',
 'NBL12',
 'NBL09',
 'HN31',
 'NBL22',
 'HN29',
 'HN22',
 'HN35',
 'NBL16',
 'NBL18',
 'HN37',
 'NBL08',
 'HN38',
 'HN13',
 'HN12',
 'HN34',
 'HN36',
 'HN44',
 'NBL23',
 'HN24',
 'HN01',
 'NBL01',
 'HN26',
 'NBL10',
 'NBL20',
 'NBL19']

In [52]:
set(df_dictCode["Code"]) - set(DF_A["Code"])

{'HN03',
 'HN04',
 'HN05',
 'HN06',
 'HN07',
 'HN08',
 'HN09',
 'HN10',
 'HN11',
 'NBL04',
 'NBL05',
 'NBL06',
 'NBL07'}

In [53]:
set(DF_A["Code"]) - set(df_dictCode["Code"])

{'NBL00', 'NBL01'}

In [57]:
'''
2. DataFrame for summarize cap code&classes
'''

df_patien = pd.read_csv("/home/kannika/code/Rheology2023/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-SummarizeFolder_Fold.csv")
print(df_patien.shape)
df_patien.head()

(53, 6)


,Unnamed: 0.1,Unnamed: 0,classes,Code,NumFolder,fold
0,0,0,No_Splenectomy,HN38,12,1
1,1,3,No_Splenectomy,HN35,8,1
2,2,11,No_Splenectomy,HN43,8,1
3,3,2,No_Splenectomy,HN33,8,2
4,4,13,No_Splenectomy,HN34,8,2


In [63]:
#df_patien.info()

In [59]:
## 1. Create Dict
lst_Code = df_patien["Code"].tolist()
lst_fold = df_patien["fold"].tolist()
dict_code = zip(lst_Code, lst_fold)
output_dict = dict(dict_code)
print(output_dict)

{'HN38': 1, 'HN35': 1, 'HN43': 1, 'HN33': 2, 'HN34': 2, 'HN14': 2, 'HN21': 3, 'HN20': 3, 'HN22': 3, 'HN41': 4, 'HN42': 4, 'HN30': 4, 'HN31': 5, 'HN18': 5, 'HN15': 5, 'HN29': 6, 'HN26': 6, 'HN17': 1, 'HN13': 1, 'HN12': 1, 'HN32': 2, 'HN19': 2, 'HN28': 2, 'HN37': 3, 'HN25': 3, 'HN02': 3, 'HN24': 4, 'HN40': 4, 'HN27': 4, 'HN16': 5, 'HN36': 5, 'HN44': 5, 'HN01': 6, 'HN39': 6, 'HN23': 6, 'NBL15': 1, 'NBL14': 1, 'NBL23': 1, 'NBL01': 2, 'NBL00': 2, 'NBL19': 2, 'NBL20': 3, 'NBL21': 3, 'NBL16': 3, 'NBL10': 4, 'NBL08': 4, 'NBL13': 4, 'NBL11': 5, 'NBL22': 5, 'NBL12': 5, 'NBL09': 6, 'NBL18': 6, 'NBL17': 6}


In [64]:
## 2. Map fold
df_dictCode["fold"]= df_dictCode['Code'].map(output_dict)
#df_dictCode["fold"] = df_dictCode["fold"].astype('int')
print(df_dictCode.shape)
df_dictCode.head()

(64, 6)


,Code,MCV,MCH,Hb,sheet,fold
0,HN01,75.8,23.1,6.0,Splenectomy,6.0
1,HN02,73.5,22.4,7.2,Splenectomy,3.0
2,HN03,77.3,24.3,9.0,Splenectomy,NaN
3,HN04,79.8,25.6,7.6,Splenectomy,NaN
4,HN06,78.5,25.9,8.2,Splenectomy,NaN


In [88]:
64-53

11

In [67]:
df_dictCode.to_csv("/home/kannika/code/Rheology2023/Rheology_Blood/All_lab_MCV-MCH-Hb_Fold_NAN.csv")

In [66]:
df_dictCode_ = df_dictCode[df_dictCode['fold'].notna()]
df_dictCode_["fold"] = df_dictCode_["fold"].astype('int')
print(df_dictCode_.shape)
df_dictCode_

(51, 6)


/tmp/ipykernel_6311/774765495.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dictCode_["fold"] = df_dictCode_["fold"].astype('int')


,Code,MCV,MCH,Hb,sheet,fold
0,HN01,75.8,23.1,6.0,Splenectomy,6
1,HN02,73.5,22.4,7.2,Splenectomy,3
6,HN12,75.4,24.1,9.8,Splenectomy,1
7,HN13,72.6,23.4,5.8,Splenectomy,1
8,HN16,76.0,24.7,10.6,Splenectomy,5
9,HN17,75.0,25.0,7.1,Splenectomy,1
10,HN19,81.3,25.9,8.9,Splenectomy,2
11,HN23,85.9,27.0,5.0,Splenectomy,6
12,HN24,71.4,19.6,7.4,Splenectomy,4
13,HN25,84.2,26.7,7.6,Splenectomy,3


In [90]:
set_fold = list(set(df_dictCode_["fold"]))
set_fold

[1, 2, 3, 4, 5, 6]

In [68]:
df_dictCode.to_csv("/home/kannika/code/Rheology2023/Rheology_Blood/All_lab_MCV-MCH-Hb_Fold_NotNAN.csv")

---------------------------------------------------------------------------------------------------------------------------------

In [71]:
df_2dFFT = pd.read_csv("/home/kannika/code/Rheology2023/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-2dFFTdataset-3channels-6Fold.csv")
print("Fold:", set(df_2dFFT['fold']))
print(df_2dFFT.shape)
df_2dFFT.head()

Fold: {1, 2, 3, 4, 5, 6}
(538, 12)


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary
0,0,0,HN01_D0_30HZ_20XINF_UWELL_20221027_154439_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154439,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1
1,1,1,HN01_D0_30HZ_20XINF_UWELL_20221027_154831_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154831,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1
2,2,2,HN01_D0_30HZ_20XINF_UWELL_20221027_155220_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155220,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1
3,3,3,HN01_D0_30HZ_20XINF_UWELL_20221027_155642_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155642,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1
4,4,4,HN01_D0_30HZ_20XINF_UWELL_20221027_160124_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_160124,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1


In [75]:
df_ML = df_dictCode[['Code', 'MCV', 'MCH', 'Hb']]
df_ML

,Code,MCV,MCH,Hb
0,HN01,75.8,23.1,6.0
1,HN02,73.5,22.4,7.2
2,HN03,77.3,24.3,9.0
3,HN04,79.8,25.6,7.6
4,HN06,78.5,25.9,8.2
...,...,...,...,...
59,NBL19,89.5,27.8,14.3
60,NBL20,95.3,31.7,14.7
61,NBL21,74.9,23.9,14.5
62,NBL22,75.1,24.7,15.2


In [76]:
df_outer = pd.merge(df_2dFFT, df_ML, on='Code', how='outer')
print(df_outer.shape)
df_outer.head()

(551, 15)


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,0.0,0.0,HN01_D0_30HZ_20XINF_UWELL_20221027_154439_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154439,SSD_Backup,HN,HN01,Splenectomy,6.0,"[2, 5, 7]",1.0,75.8,23.1,6.0
1,1.0,1.0,HN01_D0_30HZ_20XINF_UWELL_20221027_154831_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154831,SSD_Backup,HN,HN01,Splenectomy,6.0,"[2, 5, 7]",1.0,75.8,23.1,6.0
2,2.0,2.0,HN01_D0_30HZ_20XINF_UWELL_20221027_155220_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155220,SSD_Backup,HN,HN01,Splenectomy,6.0,"[2, 5, 7]",1.0,75.8,23.1,6.0
3,3.0,3.0,HN01_D0_30HZ_20XINF_UWELL_20221027_155642_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155642,SSD_Backup,HN,HN01,Splenectomy,6.0,"[2, 5, 7]",1.0,75.8,23.1,6.0
4,4.0,4.0,HN01_D0_30HZ_20XINF_UWELL_20221027_160124_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_160124,SSD_Backup,HN,HN01,Splenectomy,6.0,"[2, 5, 7]",1.0,75.8,23.1,6.0


In [77]:
df_outer

,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,0.0,0.0,HN01_D0_30HZ_20XINF_UWELL_20221027_154439_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154439,SSD_Backup,HN,HN01,Splenectomy,6.0,"[2, 5, 7]",1.0,75.8,23.1,6.0
1,1.0,1.0,HN01_D0_30HZ_20XINF_UWELL_20221027_154831_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154831,SSD_Backup,HN,HN01,Splenectomy,6.0,"[2, 5, 7]",1.0,75.8,23.1,6.0
2,2.0,2.0,HN01_D0_30HZ_20XINF_UWELL_20221027_155220_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155220,SSD_Backup,HN,HN01,Splenectomy,6.0,"[2, 5, 7]",1.0,75.8,23.1,6.0
3,3.0,3.0,HN01_D0_30HZ_20XINF_UWELL_20221027_155642_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155642,SSD_Backup,HN,HN01,Splenectomy,6.0,"[2, 5, 7]",1.0,75.8,23.1,6.0
4,4.0,4.0,HN01_D0_30HZ_20XINF_UWELL_20221027_160124_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_160124,SSD_Backup,HN,HN01,Splenectomy,6.0,"[2, 5, 7]",1.0,75.8,23.1,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HN11,NaN,NaN,NaN,NaN,59.5,19.0,8.5
547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NBL04,NaN,NaN,NaN,NaN,76.9,23.3,10.6
548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NBL05,NaN,NaN,NaN,NaN,90.6,29.8,13.9
549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NBL06,NaN,NaN,NaN,NaN,94.1,30.4,13.8


In [78]:
df_outer_notNAN = df_outer[df_outer['fold'].notna()]
df_outer_notNAN["fold"] = df_outer_notNAN["fold"].astype('int')
print(df_outer_notNAN.shape)
df_outer_notNAN

(538, 15)


/tmp/ipykernel_6311/270161573.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_outer_notNAN["fold"] = df_outer_notNAN["fold"].astype('int')


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,0.0,0.0,HN01_D0_30HZ_20XINF_UWELL_20221027_154439_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154439,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1.0,75.8,23.1,6.0
1,1.0,1.0,HN01_D0_30HZ_20XINF_UWELL_20221027_154831_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154831,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1.0,75.8,23.1,6.0
2,2.0,2.0,HN01_D0_30HZ_20XINF_UWELL_20221027_155220_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155220,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1.0,75.8,23.1,6.0
3,3.0,3.0,HN01_D0_30HZ_20XINF_UWELL_20221027_155642_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155642,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1.0,75.8,23.1,6.0
4,4.0,4.0,HN01_D0_30HZ_20XINF_UWELL_20221027_160124_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_160124,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1.0,75.8,23.1,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,533.0,533.0,NBL23_D0_30HZ_20XINF_UWELL_25660221_150600_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL23_D0_30HZ_20XINF_UWELL_25660221_150600,SSD_Backup,NBL,NBL23,Normal,1,"[2, 5, 7]",0.0,66.5,20.3,13
534,534.0,534.0,NBL23_D0_30HZ_20XINF_UWELL_25660221_150802_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL23_D0_30HZ_20XINF_UWELL_25660221_150802,SSD_Backup,NBL,NBL23,Normal,1,"[2, 5, 7]",0.0,66.5,20.3,13
535,535.0,535.0,NBL23_D0_30HZ_20XINF_UWELL_25660221_151001_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL23_D0_30HZ_20XINF_UWELL_25660221_151001,SSD_Backup,NBL,NBL23,Normal,1,"[2, 5, 7]",0.0,66.5,20.3,13
536,536.0,536.0,NBL23_D0_30HZ_20XINF_UWELL_25660221_151246_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL23_D0_30HZ_20XINF_UWELL_25660221_151246,SSD_Backup,NBL,NBL23,Normal,1,"[2, 5, 7]",0.0,66.5,20.3,13


In [79]:
df_outer_notNAN.isnull().values.any()

True

In [95]:
df_outer_notNAN_ = df_outer_notNAN[df_outer_notNAN['MCV'].notna()]
print(df_outer_notNAN_.shape)
df_outer_notNAN_ = df_outer_notNAN_.reset_index(drop=True)
df_outer_notNAN_.head()

(499, 15)


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,0.0,0.0,HN01_D0_30HZ_20XINF_UWELL_20221027_154439_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154439,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1.0,75.8,23.1,6.0
1,1.0,1.0,HN01_D0_30HZ_20XINF_UWELL_20221027_154831_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154831,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1.0,75.8,23.1,6.0
2,2.0,2.0,HN01_D0_30HZ_20XINF_UWELL_20221027_155220_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155220,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1.0,75.8,23.1,6.0
3,3.0,3.0,HN01_D0_30HZ_20XINF_UWELL_20221027_155642_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155642,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1.0,75.8,23.1,6.0
4,4.0,4.0,HN01_D0_30HZ_20XINF_UWELL_20221027_160124_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_160124,SSD_Backup,HN,HN01,Splenectomy,6,"[2, 5, 7]",1.0,75.8,23.1,6.0


In [104]:
538-499

39

In [96]:
## Check
def _(DataSet):
    lst_fold = list(set(DataSet["fold"]))
    lst_fold.sort()
    for j in range(len(lst_fold)):
        df_classes = DataSet[DataSet["fold"]==lst_fold[j]]
        Class_folder = list(set(df_classes["image_folder"]))
        lst_folder = list(set(df_classes["fold"]))
        print(f"[INFO]: [{j+1}].  Fold {lst_fold[j]} With Shape ==> {len(Class_folder)} Folder, {df_classes.shape}")
        print("Check 6-Fold With Class:")
        lst_cls = list(set(df_classes['subclass']))
        print(f"Number of Classes {len(lst_cls)} Class ==> {lst_cls}")
        for m in lst_cls:
            df_f =  df_classes[df_classes["subclass"]==m]
            list_folder = list(set(df_f["image_folder"]))
            list_code =  list(set(df_f["Code"]))
            print(f"[INFO]:  Classse {m} ==> With {len(list_folder)} Folder, {df_f.shape[0]} images!! ==> {len(list_code)} PerSon!!")
        print("="*100)
    
    return

In [97]:
_(df_outer_notNAN_)

[INFO]: [1].  Fold 1 With Shape ==> 92 Folder, (92, 15)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Splenectomy', 'Normal', 'No_Splenectomy']
[INFO]:  Classse Splenectomy ==> With 32 Folder, 32 images!! ==> 3 PerSon!!
[INFO]:  Classse Normal ==> With 32 Folder, 32 images!! ==> 3 PerSon!!
[INFO]:  Classse No_Splenectomy ==> With 28 Folder, 28 images!! ==> 3 PerSon!!
[INFO]: [2].  Fold 2 With Shape ==> 56 Folder, (56, 15)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Splenectomy', 'Normal', 'No_Splenectomy']
[INFO]:  Classse Splenectomy ==> With 24 Folder, 24 images!! ==> 3 PerSon!!
[INFO]:  Classse Normal ==> With 8 Folder, 8 images!! ==> 1 PerSon!!
[INFO]:  Classse No_Splenectomy ==> With 24 Folder, 24 images!! ==> 3 PerSon!!
[INFO]: [3].  Fold 3 With Shape ==> 88 Folder, (88, 15)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Splenectomy', 'Normal', 'No_Splenectomy']
[INFO]:  Classse Splenectomy ==> With 26 Folder, 26 images!! ==> 3 PerSon!!
[INFO]:  

In [109]:
t = df_outer_notNAN_[df_outer_notNAN_["fold"]==2.0]
t = t[t["subclass"]=="Normal"]
print(t.shape)
t.head()

(8, 15)


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
446,485.0,485.0,NBL19_D0_30HZ_20XINF_UWELL_25660117_141605_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL19_D0_30HZ_20XINF_UWELL_25660117_141605,SSD_Backup,NBL,NBL19,Normal,2,"[2, 5, 7]",0.0,89.5,27.8,14.3
447,486.0,486.0,NBL19_D0_30HZ_20XINF_UWELL_25660117_141805_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL19_D0_30HZ_20XINF_UWELL_25660117_141805,SSD_Backup,NBL,NBL19,Normal,2,"[2, 5, 7]",0.0,89.5,27.8,14.3
448,487.0,487.0,NBL19_D0_30HZ_20XINF_UWELL_25660117_142019_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL19_D0_30HZ_20XINF_UWELL_25660117_142019,SSD_Backup,NBL,NBL19,Normal,2,"[2, 5, 7]",0.0,89.5,27.8,14.3
449,488.0,488.0,NBL19_D0_30HZ_20XINF_UWELL_25660117_142230_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL19_D0_30HZ_20XINF_UWELL_25660117_142230,SSD_Backup,NBL,NBL19,Normal,2,"[2, 5, 7]",0.0,89.5,27.8,14.3
450,489.0,489.0,NBL19_D0_30HZ_20XINF_UWELL_25660117_142455_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL19_D0_30HZ_20XINF_UWELL_25660117_142455,SSD_Backup,NBL,NBL19,Normal,2,"[2, 5, 7]",0.0,89.5,27.8,14.3


In [111]:
set(t["Code"])

{'NBL19'}

In [81]:
df_outerNAN = df_outer_notNAN[df_outer_notNAN.isna().any(axis=1)]
print(df_outerNAN.shape)
df_outerNAN

(39, 15)


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
337,337.0,337.0,NBL00_D0_30HZ_20XINF_UWELL_20221031_101419_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL00_D0_30HZ_20XINF_UWELL_20221031_101419,SSD_Backup,NBL,NBL00,Normal,2,"[2, 5, 7]",0.0,NaN,NaN,NaN
338,338.0,338.0,NBL00_D0_30HZ_20XINF_UWELL_20221031_102006_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL00_D0_30HZ_20XINF_UWELL_20221031_102006,SSD_Backup,NBL,NBL00,Normal,2,"[2, 5, 7]",0.0,NaN,NaN,NaN
339,339.0,339.0,NBL00_D0_30HZ_20XINF_UWELL_20221031_102621_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL00_D0_30HZ_20XINF_UWELL_20221031_102621,SSD_Backup,NBL,NBL00,Normal,2,"[2, 5, 7]",0.0,NaN,NaN,NaN
340,340.0,340.0,NBL00_D0_30HZ_20XINF_UWELL_20221031_103325_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL00_D0_30HZ_20XINF_UWELL_20221031_103325,SSD_Backup,NBL,NBL00,Normal,2,"[2, 5, 7]",0.0,NaN,NaN,NaN
341,341.0,341.0,NBL00_D0_30HZ_20XINF_UWELL_20221031_104009_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL00_D0_30HZ_20XINF_UWELL_20221031_104009,SSD_Backup,NBL,NBL00,Normal,2,"[2, 5, 7]",0.0,NaN,NaN,NaN
342,342.0,342.0,NBL00_D0_30HZ_20XINF_UWELL_20221031_104541_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL00_D0_30HZ_20XINF_UWELL_20221031_104541,SSD_Backup,NBL,NBL00,Normal,2,"[2, 5, 7]",0.0,NaN,NaN,NaN
343,343.0,343.0,NBL00_D0_30HZ_20XINF_UWELL_20221031_105045_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL00_D0_30HZ_20XINF_UWELL_20221031_105045,SSD_Backup,NBL,NBL00,Normal,2,"[2, 5, 7]",0.0,NaN,NaN,NaN
344,344.0,344.0,NBL00_D0_30HZ_20XINF_UWELL_20221031_105609_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL00_D0_30HZ_20XINF_UWELL_20221031_105609,SSD_Backup,NBL,NBL00,Normal,2,"[2, 5, 7]",0.0,NaN,NaN,NaN
345,345.0,345.0,NBL00_D0_30HZ_20XINF_UWELL_20221031_110403_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL00_D0_30HZ_20XINF_UWELL_20221031_110403,SSD_Backup,NBL,NBL00,Normal,2,"[2, 5, 7]",0.0,NaN,NaN,NaN
346,346.0,346.0,NBL00_D0_30HZ_20XINF_UWELL_20221031_110820_dt2...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,NBL00_D0_30HZ_20XINF_UWELL_20221031_110820,SSD_Backup,NBL,NBL00,Normal,2,"[2, 5, 7]",0.0,NaN,NaN,NaN


In [82]:
print(set(df_outerNAN["fold"]))

{2}


In [83]:
print(set(df_outerNAN["Code"]))

{'NBL01', 'NBL00'}


------------------------------------------------------------------------------------------------------------------------------

In [98]:
df_outer_A = pd.merge(DF_A , df_ML, on='Code', how='outer')
print(df_outer_A.shape)
df_outer_A.head()

(301, 18)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,0.0,0.0,0.0,237.0,237.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162433_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162433,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
1,1.0,1.0,1.0,234.0,234.0,HN35_D0_30HZ_20XINF_UWELL_25660131_161754_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_161754,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
2,4.0,4.0,4.0,239.0,239.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
3,8.0,8.0,8.0,240.0,240.0,HN35_D0_30HZ_20XINF_UWELL_25660131_163058_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_163058,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
4,2.0,2.0,2.0,324.0,324.0,HN43_D0_30HZ_20XINF_UWELL_25660330_153259_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN43_D0_30HZ_20XINF_UWELL_25660330_153259,SSD_Backup,HN,HN43,No_Splenectomy,1.0,"[2, 5, 7]",1.0,57.4,17.7,4.4


In [100]:
df_outer_A_notNAN = df_outer_A[df_outer_A['fold'].notna()]
df_outer_A_notNAN_ = df_outer_A_notNAN[df_outer_A_notNAN['MCV'].notna()]
print(df_outer_A_notNAN_.shape)
df_outer_A_notNAN_ = df_outer_A_notNAN_.reset_index(drop=True)
df_outer_A_notNAN_.head()

(268, 18)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,0.0,0.0,0.0,237.0,237.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162433_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162433,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
1,1.0,1.0,1.0,234.0,234.0,HN35_D0_30HZ_20XINF_UWELL_25660131_161754_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_161754,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
2,4.0,4.0,4.0,239.0,239.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
3,8.0,8.0,8.0,240.0,240.0,HN35_D0_30HZ_20XINF_UWELL_25660131_163058_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_163058,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
4,2.0,2.0,2.0,324.0,324.0,HN43_D0_30HZ_20XINF_UWELL_25660330_153259_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN43_D0_30HZ_20XINF_UWELL_25660330_153259,SSD_Backup,HN,HN43,No_Splenectomy,1.0,"[2, 5, 7]",1.0,57.4,17.7,4.4


In [102]:
df_outer_A_notNAN_.isnull().values.any()

False

In [103]:
_(df_outer_A_notNAN_)

[INFO]: [1].  Fold 1.0 With Shape ==> 48 Folder, (48, 18)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Splenectomy', 'Normal', 'No_Splenectomy']
[INFO]:  Classse Splenectomy ==> With 12 Folder, 12 images!! ==> 3 PerSon!!
[INFO]:  Classse Normal ==> With 24 Folder, 24 images!! ==> 3 PerSon!!
[INFO]:  Classse No_Splenectomy ==> With 12 Folder, 12 images!! ==> 3 PerSon!!
[INFO]: [2].  Fold 2.0 With Shape ==> 28 Folder, (28, 18)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Splenectomy', 'Normal', 'No_Splenectomy']
[INFO]:  Classse Splenectomy ==> With 12 Folder, 12 images!! ==> 3 PerSon!!
[INFO]:  Classse Normal ==> With 4 Folder, 4 images!! ==> 1 PerSon!!
[INFO]:  Classse No_Splenectomy ==> With 12 Folder, 12 images!! ==> 3 PerSon!!
[INFO]: [3].  Fold 3.0 With Shape ==> 48 Folder, (48, 18)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Splenectomy', 'Normal', 'No_Splenectomy']
[INFO]:  Classse Splenectomy ==> With 12 Folder, 12 images!! ==> 3 PerSon!!
[IN

In [112]:
tt = df_outer_A_notNAN[df_outer_A_notNAN["fold"]==2.0]
tt = tt[tt["subclass"]=="Normal"]
print(tt.shape)
tt.head()

(24, 18)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
60,60.0,60.0,60.0,367.0,367.0,NBL01_D0_30HZ_20XINF_UWELL_20221031_140347_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL01_D0_30HZ_20XINF_UWELL_20221031_140347,SSD_Backup,NBL,NBL01,Normal,2.0,"[2, 5, 7]",0.0,NaN,NaN,NaN
61,61.0,61.0,61.0,366.0,366.0,NBL01_D0_30HZ_20XINF_UWELL_20221031_135941_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL01_D0_30HZ_20XINF_UWELL_20221031_135941,SSD_Backup,NBL,NBL01,Normal,2.0,"[2, 5, 7]",0.0,NaN,NaN,NaN
62,68.0,68.0,68.0,359.0,359.0,NBL01_D0_30HZ_20XINF_UWELL_20221031_133117_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL01_D0_30HZ_20XINF_UWELL_20221031_133117,SSD_Backup,NBL,NBL01,Normal,2.0,"[2, 5, 7]",0.0,NaN,NaN,NaN
63,70.0,70.0,70.0,371.0,371.0,NBL01_D0_30HZ_20XINF_UWELL_20221031_142403_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL01_D0_30HZ_20XINF_UWELL_20221031_142403,SSD_Backup,NBL,NBL01,Normal,2.0,"[2, 5, 7]",0.0,NaN,NaN,NaN
64,71.0,71.0,71.0,364.0,364.0,NBL01_D0_30HZ_20XINF_UWELL_20221031_135036_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL01_D0_30HZ_20XINF_UWELL_20221031_135036,SSD_Backup,NBL,NBL01,Normal,2.0,"[2, 5, 7]",0.0,NaN,NaN,NaN


In [113]:
print(set(tt["fold"]))

{2.0}


In [114]:
print(set(tt["Code"]))

{'NBL01', 'NBL19', 'NBL00'}


In [115]:
df_patien = pd.read_csv("/home/kannika/code/Rheology2023/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-SummarizeFolder.csv")
print(df_patien.shape)
df_patien.head()

(53, 4)


,Unnamed: 0,classes,Code,NumFolder
0,0,No_Splenectomy,HN38,12
1,1,No_Splenectomy,HN18,8
2,2,No_Splenectomy,HN33,8
3,3,No_Splenectomy,HN35,8
4,4,No_Splenectomy,HN22,12


In [117]:
patien_Normal = df_patien[df_patien["classes"]=="Normal"]
patien_NBL01 = df_patien[df_patien["Code"]== 'NBL01']
print(patien_NBL01)
patien_NBL00 = df_patien[df_patien["Code"]== 'NBL00']
print(patien_NBL00)

    Unnamed: 0 classes   Code  NumFolder
18          18  Normal  NBL01         19
    Unnamed: 0 classes   Code  NumFolder
17          17  Normal  NBL00         20


In [119]:
print(patien_Normal.shape)
patien_Normal

(18, 4)


,Unnamed: 0,classes,Code,NumFolder
17,17,Normal,NBL00,20
18,18,Normal,NBL01,19
19,19,Normal,NBL18,8
20,20,Normal,NBL11,12
21,21,Normal,NBL20,13
22,22,Normal,NBL10,8
23,23,Normal,NBL14,8
24,24,Normal,NBL22,12
25,25,Normal,NBL15,12
26,26,Normal,NBL17,8


In [120]:
DF_B = pd.read_csv('/home/kannika/code/Rheology2023/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-2dFFTdataset-3channels-3Fold-split3class.csv')
df_outer_B = pd.merge(DF_B , df_ML, on='Code', how='outer')
print(df_outer_B.shape)
df_outer_B.head()

(301, 18)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,0.0,0.0,0.0,239.0,239.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
1,2.0,2.0,2.0,240.0,240.0,HN35_D0_30HZ_20XINF_UWELL_25660131_163058_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_163058,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
2,5.0,5.0,5.0,234.0,234.0,HN35_D0_30HZ_20XINF_UWELL_25660131_161754_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_161754,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
3,14.0,14.0,14.0,236.0,236.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162205_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162205,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
4,15.0,15.0,15.0,238.0,238.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162641_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162641,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1


In [121]:
df_outer_B_notNAN = df_outer_B[df_outer_B['fold'].notna()]
df_outer_B_notNAN_ = df_outer_B_notNAN[df_outer_B_notNAN['MCV'].notna()]
print(df_outer_B_notNAN_.shape)
df_outer_B_notNAN_ = df_outer_B_notNAN_.reset_index(drop=True)
df_outer_B_notNAN_.head()

(274, 18)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,0.0,0.0,0.0,239.0,239.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
1,2.0,2.0,2.0,240.0,240.0,HN35_D0_30HZ_20XINF_UWELL_25660131_163058_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_163058,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
2,5.0,5.0,5.0,234.0,234.0,HN35_D0_30HZ_20XINF_UWELL_25660131_161754_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_161754,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
3,14.0,14.0,14.0,236.0,236.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162205_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162205,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1
4,15.0,15.0,15.0,238.0,238.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162641_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162641,SSD_Backup,HN,HN35,No_Splenectomy,1.0,"[2, 5, 7]",1.0,74.3,23.4,7.1


In [122]:
_(df_outer_B_notNAN_)

[INFO]: [1].  Fold 1.0 With Shape ==> 82 Folder, (82, 18)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Splenectomy', 'Normal', 'No_Splenectomy']
[INFO]:  Classse Splenectomy ==> With 32 Folder, 32 images!! ==> 6 PerSon!!
[INFO]:  Classse Normal ==> With 18 Folder, 18 images!! ==> 4 PerSon!!
[INFO]:  Classse No_Splenectomy ==> With 32 Folder, 32 images!! ==> 6 PerSon!!
[INFO]: [2].  Fold 2.0 With Shape ==> 96 Folder, (96, 18)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Splenectomy', 'Normal', 'No_Splenectomy']
[INFO]:  Classse Splenectomy ==> With 32 Folder, 32 images!! ==> 6 PerSon!!
[INFO]:  Classse Normal ==> With 32 Folder, 32 images!! ==> 6 PerSon!!
[INFO]:  Classse No_Splenectomy ==> With 32 Folder, 32 images!! ==> 6 PerSon!!
[INFO]: [3].  Fold 3.0 With Shape ==> 96 Folder, (96, 18)
Check 6-Fold With Class:
Number of Classes 3 Class ==> ['Splenectomy', 'Normal', 'No_Splenectomy']
[INFO]:  Classse Splenectomy ==> With 32 Folder, 32 images!! ==> 6 PerSon!!
[

In [86]:
a = DF_A[DF_A["Code"]=="NBL01"]
print(a.shape)
a.head()

(11, 15)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary
60,60,60,60,367,367,NBL01_D0_30HZ_20XINF_UWELL_20221031_140347_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL01_D0_30HZ_20XINF_UWELL_20221031_140347,SSD_Backup,NBL,NBL01,Normal,2,"[2, 5, 7]",0
61,61,61,61,366,366,NBL01_D0_30HZ_20XINF_UWELL_20221031_135941_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL01_D0_30HZ_20XINF_UWELL_20221031_135941,SSD_Backup,NBL,NBL01,Normal,2,"[2, 5, 7]",0
68,68,68,68,359,359,NBL01_D0_30HZ_20XINF_UWELL_20221031_133117_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL01_D0_30HZ_20XINF_UWELL_20221031_133117,SSD_Backup,NBL,NBL01,Normal,2,"[2, 5, 7]",0
70,70,70,70,371,371,NBL01_D0_30HZ_20XINF_UWELL_20221031_142403_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL01_D0_30HZ_20XINF_UWELL_20221031_142403,SSD_Backup,NBL,NBL01,Normal,2,"[2, 5, 7]",0
71,71,71,71,364,364,NBL01_D0_30HZ_20XINF_UWELL_20221031_135036_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL01_D0_30HZ_20XINF_UWELL_20221031_135036,SSD_Backup,NBL,NBL01,Normal,2,"[2, 5, 7]",0


In [87]:
b = DF_A[DF_A["Code"]=="NBL00"]
print(b.shape)
b.head()

(9, 15)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary
63,63,63,63,354,354,NBL00_D0_30HZ_20XINF_UWELL_20221031_114116_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL00_D0_30HZ_20XINF_UWELL_20221031_114116,SSD_Backup,NBL,NBL00,Normal,2,"[2, 5, 7]",0
64,64,64,64,351,351,NBL00_D0_30HZ_20XINF_UWELL_20221031_112909_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL00_D0_30HZ_20XINF_UWELL_20221031_112909,SSD_Backup,NBL,NBL00,Normal,2,"[2, 5, 7]",0
65,65,65,65,355,355,NBL00_D0_30HZ_20XINF_UWELL_20221031_114531_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL00_D0_30HZ_20XINF_UWELL_20221031_114531,SSD_Backup,NBL,NBL00,Normal,2,"[2, 5, 7]",0
66,66,66,66,349,349,NBL00_D0_30HZ_20XINF_UWELL_20221031_112057_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL00_D0_30HZ_20XINF_UWELL_20221031_112057,SSD_Backup,NBL,NBL00,Normal,2,"[2, 5, 7]",0
67,67,67,67,356,356,NBL00_D0_30HZ_20XINF_UWELL_20221031_114933_dt2...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,NBL00_D0_30HZ_20XINF_UWELL_20221031_114933,SSD_Backup,NBL,NBL00,Normal,2,"[2, 5, 7]",0


-------------------------------------------------------

# Thalassemia

In [2]:
df_dictCode = pd.read_csv("/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_TrainML_3Fold_EMclass.csv")
print(df_dictCode.shape)
## NotNaN
#df_dictCode_ = df_dictCode[df_dictCode['fold'].notna()]
#df_dictCode_["fold"] = df_dictCode_["fold"].astype('int')
print(df_dictCode.shape)
df_dictCode.head()

(33, 11)
(33, 11)


,Unnamed: 0.1,Unnamed: 0,Code,classes,subclass,classes_binary,MCV,MCH,Hb,typeBEvsBM,fold
0,0,42,HN44,HN,Splenectomy,1.0,61.7,19.7,7.0,E,1
1,1,17,HN21,HN,No_Splenectomy,1.0,64.8,19.7,6.5,E,1
2,2,8,HN13,HN,Splenectomy,1.0,72.6,23.4,5.8,E,1
3,3,43,HN29,HN,No_Splenectomy,1.0,64.2,19.1,6.6,E,1
4,4,50,HN39,HN,Splenectomy,1.0,76.4,22.5,6.4,E,1


In [32]:
for j in range(len(list_feature)):
#df_class.head()
    print(f"[INFO]: Fetures ==> {list_feature[j]}")
    list_j = list(df_dictCode[list_feature[j]])
    print("-"*125)
    print(f"[INFO]: Min ==> {min(list_j)}")
    print(f"[INFO]: Max ==> {max(list_j)}")
    print(f"[INFO]: Mean ==> {mean(list_j)}")
    print('*'*125)

[INFO]: Fetures ==> MCV
-----------------------------------------------------------------------------------------------------------------------------
[INFO]: Min ==> 55.3
[INFO]: Max ==> 85.9
[INFO]: Mean ==> 70.8181818181818
*****************************************************************************************************************************
[INFO]: Fetures ==> MCH
-----------------------------------------------------------------------------------------------------------------------------
[INFO]: Min ==> 15.6
[INFO]: Max ==> 27.0
[INFO]: Mean ==> 22.027272727272727
*****************************************************************************************************************************
[INFO]: Fetures ==> Hb
-----------------------------------------------------------------------------------------------------------------------------
[INFO]: Min ==> 4.4
[INFO]: Max ==> 10.6
[INFO]: Mean ==> 7.251515151515152
*********************************************************************

In [16]:
df_dictCode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0.1    33 non-null     int64  
 1   Unnamed: 0      33 non-null     int64  
 2   Code            33 non-null     object 
 3   classes         33 non-null     object 
 4   subclass        33 non-null     object 
 5   classes_binary  33 non-null     float64
 6   MCV             33 non-null     float64
 7   MCH             33 non-null     float64
 8   Hb              33 non-null     float64
 9   typeBEvsBM      33 non-null     object 
 10  fold            33 non-null     int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 3.0+ KB


In [4]:
subtype = list(set(df_dictCode['typeBEvsBM']))
subtype

['E', 'M']

In [5]:
list_feature = ['MCV', 'MCH', 'Hb']

In [6]:
from numpy import mean
import numpy as np

for k in range(len(subtype)):
    df_subtype = df_dictCode[df_dictCode["typeBEvsBM"]==subtype[k]]
    print(f"[INFO]: FOR Subtypes : {subtype[k]}")
    print(df_subtype.shape)
    print('='*125)
    for j in range(len(list_feature)):
    #df_class.head()
        print(f"[INFO]: Fetures ==> {list_feature[j]}")
        list_j = list(df_subtype[list_feature[j]])
        print("-"*125)
        print(f"[INFO]: Min ==> {min(list_j)}")
        print(f"[INFO]: Max ==> {max(list_j)}")
        print(f"[INFO]: Mean ==> {mean(list_j)}")
        print(f"[INFO]: std ==> {np.std(list_j)}")
        print('*'*125)

[INFO]: FOR Subtypes : E
(24, 11)
[INFO]: Fetures ==> MCV
-----------------------------------------------------------------------------------------------------------------------------
[INFO]: Min ==> 55.3
[INFO]: Max ==> 85.9
[INFO]: Mean ==> 70.90416666666665
[INFO]: std ==> 7.217714271537462
*****************************************************************************************************************************
[INFO]: Fetures ==> MCH
-----------------------------------------------------------------------------------------------------------------------------
[INFO]: Min ==> 15.6
[INFO]: Max ==> 27.0
[INFO]: Mean ==> 22.108333333333334
[INFO]: std ==> 2.6609704787706474
*****************************************************************************************************************************
[INFO]: Fetures ==> Hb
-----------------------------------------------------------------------------------------------------------------------------
[INFO]: Min ==> 5.0
[INFO]: Max ==> 10.6


## Normal

In [27]:
#df_main

In [7]:
from numpy import mean

df_main = pd.read_csv("/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_TrainML_3Fold_split3class.csv")
df_Normal = df_main[df_main["subclass"]=='Normal']
print(df_Normal.shape)

for j in range(len(list_feature)):
#df_class.head()
    print(f"[INFO]: Fetures ==> {list_feature[j]}")
    list_j = list(df_Normal[list_feature[j]])
    print("-"*125)
    print(f"[INFO]: Min ==> {min(list_j)}")
    print(f"[INFO]: Max ==> {max(list_j)}")
    print(f"[INFO]: Mean ==> {mean(list_j)}")
    print(f"[INFO]: std ==> {np.std(list_j)}")
    print('*'*125)

(16, 9)
[INFO]: Fetures ==> MCV
-----------------------------------------------------------------------------------------------------------------------------
[INFO]: Min ==> 55.5
[INFO]: Max ==> 95.3
[INFO]: Mean ==> 82.375
[INFO]: std ==> 10.500922578516613
*****************************************************************************************************************************
[INFO]: Fetures ==> MCH
-----------------------------------------------------------------------------------------------------------------------------
[INFO]: Min ==> 17.9
[INFO]: Max ==> 31.7
[INFO]: Mean ==> 26.175
[INFO]: std ==> 3.629652187193699
*****************************************************************************************************************************
[INFO]: Fetures ==> Hb
-----------------------------------------------------------------------------------------------------------------------------
[INFO]: Min ==> 11.2
[INFO]: Max ==> 15.2
[INFO]: Mean ==> 13.331249999999999
[INFO]: std 

## Thalas

In [8]:
from numpy import mean

df_main = pd.read_csv("/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_TrainML_3Fold_split3class.csv")
df_Normal = df_main[df_main["subclass"]!='Normal']
print(df_Normal.shape)

for j in range(len(list_feature)):
#df_class.head()
    print(f"[INFO]: Fetures ==> {list_feature[j]}")
    list_j = list(df_Normal[list_feature[j]])
    print("-"*125)
    print(f"[INFO]: Min ==> {min(list_j)}")
    print(f"[INFO]: Max ==> {max(list_j)}")
    print(f"[INFO]: Mean ==> {mean(list_j)}")
    print(f"[INFO]: std ==> {np.std(list_j)}")
    print('*'*125)

(35, 9)
[INFO]: Fetures ==> MCV
-----------------------------------------------------------------------------------------------------------------------------
[INFO]: Min ==> 55.3
[INFO]: Max ==> 85.9
[INFO]: Mean ==> 71.25142857142858
[INFO]: std ==> 7.738193640399969
*****************************************************************************************************************************
[INFO]: Fetures ==> MCH
-----------------------------------------------------------------------------------------------------------------------------
[INFO]: Min ==> 15.6
[INFO]: Max ==> 27.0
[INFO]: Mean ==> 22.122857142857146
[INFO]: std ==> 2.7374738839496024
*****************************************************************************************************************************
[INFO]: Fetures ==> Hb
-----------------------------------------------------------------------------------------------------------------------------
[INFO]: Min ==> 4.4
[INFO]: Max ==> 10.6
[INFO]: Mean ==> 7.2285714

-----

### our dataset

In [33]:
from numpy import mean

df_main = pd.read_csv("/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_TrainML_3Fold_split3class.csv")
print(df_main.shape)

for j in range(len(list_feature)):
#df_class.head()
    print(f"[INFO]: Fetures ==> {list_feature[j]}")
    list_j = list(df_main[list_feature[j]])
    print("-"*125)
    print(f"[INFO]: Min ==> {min(list_j)}")
    print(f"[INFO]: Max ==> {max(list_j)}")
    print(f"[INFO]: Mean ==> {mean(list_j)}")
    print('*'*125)

(51, 9)
[INFO]: Fetures ==> MCV
-----------------------------------------------------------------------------------------------------------------------------
[INFO]: Min ==> 55.3
[INFO]: Max ==> 95.3
[INFO]: Mean ==> 74.74117647058824
*****************************************************************************************************************************
[INFO]: Fetures ==> MCH
-----------------------------------------------------------------------------------------------------------------------------
[INFO]: Min ==> 15.6
[INFO]: Max ==> 31.7
[INFO]: Mean ==> 23.39411764705882
*****************************************************************************************************************************
[INFO]: Fetures ==> Hb
-----------------------------------------------------------------------------------------------------------------------------
[INFO]: Min ==> 4.4
[INFO]: Max ==> 15.2
[INFO]: Mean ==> 9.14313725490196
**************************************************************